# Importing the libraries

In [1]:
import cv2
import mediapipe as mp
import pyfirmata
import numpy as np
import time
import math
from timer import Timer
import winsound as ws
from play import Play as snd

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_face_detection = mp.solutions.face_detection

In [3]:
MIDDLE_X = 70
MIDDLE_Y = 70

# Connecting to the Arduino Uno

In [4]:
port = "COM3"
board = pyfirmata.Arduino(port)

In [5]:
servo_pinX = board.get_pin('d:3:s') 
servo_pinY = board.get_pin('d:2:s')
servo_pinX.write(90)
servo_pinY.write(90)

laser1 = board.digital[8]
laser2 = board.digital[9]
laser1.write(1)
laser2.write(1)

# Helper Fucntions

In [6]:
# https://stackoverflow.com/questions/37642834/opencv-how-to-calculate-the-degreesangles-of-an-object-with-its-coordinates

w = 640 # Width of the image
h = 480 # Height of the image
fov_x = 70 # Horizontal field of view of the camera
fov_y = 50 # Vertical field of view of the camera

a_x = w/2
a_y = h/2

alpha_x = fov_x/2
alpha_y = fov_y/2

Kx = np.tan(np.deg2rad(alpha_x)) / a_x
Ky = np.tan(np.deg2rad(alpha_y)) / a_y

# Testing
p = 134
b = p - h/2
beta = np.rad2deg(np.arctan(Ky * b))
round(beta + 90)

78

In [7]:
idle_sweep_dir = True

def idle_sweep():

    global idle_sweep_dir

    servo_pinY.write(70)

    currAngleX = servo_pinX.read()


    if currAngleX >= 180:
        idle_sweep_dir = False
    elif currAngleX <= 0:
        idle_sweep_dir = True
    
    inc = 1 if idle_sweep_dir else -1

    servo_pinX.write(currAngleX + inc)




In [8]:
def move_servo(x: int, y: int) -> tuple:
    global servo_pinX, servo_pinY

    b_x = x - w/2
    b_y = y - h/2

    beta_x = np.rad2deg(np.arctan(Kx * b_x))
    beta_y = np.rad2deg(np.arctan(Ky * b_y))

    angle_x = round(180 - (beta_x + 80))
    angle_y = round(beta_y + 55)

    servo_pinY.write(angle_y)
    servo_pinX.write(angle_x)

    return angle_x, angle_y

In [9]:
def toggle_laser(turn: bool) -> None:
    global laser1, laser2
    if turn:
        laser1.write(1)
        laser2.write(0)
    else:
        laser1.write(0)
        laser2.write(1)

In [10]:
def shutdown(cap):
    ws.PlaySound(None, ws.SND_PURGE)
    cv2.destroyAllWindows()
    cap.release()
    laser1.write(0)
    laser2.write(0)
    servo_pinX.write(90)
    servo_pinY.write(130)

In [11]:
def get_shoulders(results, image):
        # Find the position of the shoulders
    right = (int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image.shape[1]),
                int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image.shape[0]))
    
    left = (int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image.shape[1]),
                    int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image.shape[0]))
    
    return right, left

In [12]:
def get_distance(right, left):
    return round(math.sqrt((right[0] - left[0])**2 + (right[1] - left[1])**2))

In [13]:
def get_middle(right, left):
    # right, left = get_shoulders(right, left)
    return (round((right[0] + left[0])/2), round((right[1] + left[1])/2))

In [23]:
def get_sounds():
    gun_sound = './sounds/gun2.wav'
    hostile_detected_sound = './sounds/hostile.wav'
    jb_sound = './sounds/joy_b.wav'
    tp_warn_sound = './sounds/warn.wav'
    last_warn = './sounds/warn2.wav'

    gun = snd(gun_sound)
    hostile_detected = snd(hostile_detected_sound)
    jb = snd(jb_sound)
    tp = snd(tp_warn_sound)
    warn = snd(last_warn)

    return gun, hostile_detected, jb, tp, warn

In [15]:
def check_surrender_pose(results):
    left_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y
    right_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y

    right_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y
    left_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y

    if left_wrist_y < left_shoulder_y and right_wrist_y < right_shoulder_y:
        return True
    return False

In [16]:
def shoot_gun(gun, laserTurn, shooting_timer, image):
    cv2.putText(image, f'Shooting: {round(shooting_timer.get_time_left())}', (50, 100),
                        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
            
    gun.play()

    laserTurn = not laserTurn
    toggle_laser(laserTurn)

    return laserTurn

# Main Function

In [27]:
cap = cv2.VideoCapture(2)

# ws, hs = 1920, 1080
# cap.set(3, ws)
# cap.set(4, hs)

servo_pinX.write(MIDDLE_X)
servo_pinY.write(MIDDLE_Y)
offsetY = 0
start_time = None
status = ''


# distances
tresspass_dist = 80
hostile_dist = 130
kill_dist = 200

gun, hostile_detected, jb, tp, warn = get_sounds()



with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    

    # To alternate lasers when shooting
    laserTurn = True

    # Is the target surrendering
    isSurrendering = True

    # Is the gun shooting
    isShooting = False

    # Is the target tresspassing (prompted to surrender)
    isTrespassing = False

    # Is the target in the safe zone
    isInsidePerimeter = False


    yield_timer = Timer(5)
    shooting_timer = Timer(3)

    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Camera not found.")
            print("Please check whether the camera is connected properly.")
            break
        
        isIdle = False
        

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # Draw the pose annotation on the image.
        # image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # mp_drawing.draw_landmarks(
        #     image,
        #     results.pose_landmarks,
        #     mp_pose.POSE_CONNECTIONS,
        #     landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        if results.pose_landmarks:

            # Pose detection
            isSurrendering = check_surrender_pose(results)

            # Measuring distance by shoulder width
            right, left = get_shoulders(results, image)
            distance = get_distance(right, left)

            cv2.putText(image, f'Distance: {distance}', (50, 150),
                        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)


            # Target is outside of the perimeter
            if distance < tresspass_dist:
                isIdle = True
                status = 'Idle'
            
            else:
                # Target is inside the perimeter - action required
                isIdle = False

                # Getting middle point
                mid_x, mid_y = get_middle(right, left)

                # Aiming at the target
                trans_x, trans_y = move_servo(mid_x, mid_y)

                # Drawing Crosshair
                cv2.circle(image, (mid_x, mid_y), 50, (0, 0, 255), 2)
                cv2.circle(image, (mid_x, mid_y),
                           15, (0, 0, 255), cv2.FILLED)

                # Immediate kill - surrendering will not be accepted
                if distance > kill_dist:
                    isShooting = True

                    if not shooting_timer.is_timer_running():
                        shooting_timer.start()

                # Command suspect to surrender
                elif distance > hostile_dist:
                    warn.play()
                    isTrespassing = True
                    status = 'Prompted to surrender'

                    if not any([isShooting, isSurrendering]):
                        if yield_timer.is_time_up():
                            isShooting = True
                            isTrespassing = False
                            shooting_timer.start()

                        if not yield_timer.is_timer_running():
                            yield_timer.start()

                        cv2.putText(image, f'Shooting in: {round(yield_timer.get_time_left())}', (50, 100),
                                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
                    
                    if isSurrendering:
                        yield_timer.clear()

                # Command suspect to stop trespassing
                elif distance > tresspass_dist:
                    status = 'Trespassing'
                    isInsidePerimeter = True
                    tp.play()
                
            

        else:
            # No target detected
            yield_timer.clear()
            shooting_timer.clear()
            isIdle = True


        if shooting_timer.is_time_up():
            isShooting = False
            gun.stop()
            shooting_timer.clear()
            yield_timer.clear()

        elif isShooting:
            status = 'Shooting'
            laserTurn = shoot_gun(gun, laserTurn, shooting_timer, image)
        else:
            laser1.write(1)
            laser2.write(1)

        cv2.putText(image, f'Surrender: {isSurrendering}', (50, 50),
                    cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
        

        cv2.putText(image, f'Status: {status.upper()}', (50, 470),
                    cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

        # Showing image
        # cv2.imshow('MediaPipe Pose', cv2.flip(image, 1)) # Flip image
        cv2.imshow('Camera Feed', image)
        if cv2.waitKey(1) & 0xFF == 27: # ESC
            break

        idle_sweep() if isIdle else None

shutdown(cap)
jb.play()

# Misc

In [16]:
servo_pinX.write(90)
servo_pinY.write(70)

In [ ]:
laser1.write(1)
laser2.write(1)
servo_pinY.write(50)

In [ ]:
cv2.destroyAllWindows()
cap.release()

In [ ]:
servo_pinX.write(70)

In [ ]:
testx = 180

while True:
    servo_pinX.write(testx)
    testx -= 1
    time.sleep(0.01)

    if testx == 0:
        testx = 180



In [ ]:
while True:
    servo_pinY.write(15)
    time.sleep(1)
    servo_pinY.write(80)
    time.sleep(1)

## Lists all the available serial ports

In [ ]:
def list_ports():
    """
    Test the ports and returns a tuple with the available ports and the ones that are working.
    """
    non_working_ports = []
    dev_port = 0
    working_ports = []
    available_ports = []
    while len(non_working_ports) < 6: # if there are more than 5 non working ports stop the testing. 
        camera = cv2.VideoCapture(dev_port)
        if not camera.isOpened():
            non_working_ports.append(dev_port)
            print("Port %s is not working." %dev_port)
        else:
            is_reading, img = camera.read()
            w = camera.get(3)
            h = camera.get(4)
            if is_reading:
                print("Port %s is working and reads images (%s x %s)" %(dev_port,h,w))
                working_ports.append(dev_port)
            else:
                print("Port %s for camera ( %s x %s) is present but does not reads." %(dev_port,h,w))
                available_ports.append(dev_port)
        dev_port +=1
    return available_ports,working_ports,non_working_ports

list_ports()

Port 0 is working and reads images (480.0 x 640.0)
Port 1 is working and reads images (480.0 x 640.0)
Port 2 is working and reads images (480.0 x 640.0)
Port 3 is working and reads images (480.0 x 640.0)
Port 4 is not working.
Port 5 is not working.
Port 6 is not working.
Port 7 is not working.
Port 8 is not working.
Port 9 is not working.


([], [0, 1, 2, 3], [4, 5, 6, 7, 8, 9])